# Biostat M280 Homework 2

**Due May 11 @ 11:59PM**

## Q1. Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Later in the course we will learn how to derive a majorization-minimization (MM) algorithm with iterative updates
$$
	v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
	w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t+1)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t+1)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}
$$
that drive the objective $L^{(t)} = L(\mathbf{V}^{(t)}, \mathbf{W}^{(t)})$ downhill. Superscript $t$ indicates iteration number. Efficiency (both speed and memory) will be the most important criterion when grading this problem.


1. Implement the algorithm with arguments: $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.
```julia
function nnmf(
    X::Matrix, 
    r::Int;
    maxiter::Int=1000, 
    tol::eltype(X)=1e-4,
    V::Matrix{eltype(X)}=rand(size(X, 1), r),
    W::Matrix{eltype(X)}=rand(r, size(X, 2))
    )
    # implementation
    # Output
    return V, W
end
```

0. Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  
Read in the [`nnmf-2429-by-361-face.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm()`](https://docs.julialang.org/en/stable/stdlib/io-network/#Base.DataFmt.readdlm-Tuple{Any,Char,Type,Char}) function, and display a couple sample images, e.g., using [ImageView.jl](https://github.com/JuliaImages/ImageView.jl) package.

0. Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt)) and stopping criterion
$$
	\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

0. Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

0. (Optional) Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in part 2.

In [50]:
# possible for efficiency: column major/ use blas or lapack/overwrite the initial V and W
function nnmf3(
    X::Matrix{T}, 
    r::Int;
    maxiter::Int=1000, 
    tol::T=1e-4,
    V::Matrix{T}=rand(T, size(X, 1), r),
    W::Matrix{T}=rand(T, r, size(X, 2))
    ) where T <: AbstractFloat
    # implementation
    m = size(X,1)
    n = size(X,2)
    # proallocate. capital letter is for matrix, lowercase "t" means matrix transpose.
    XWt = zeros(m, r)
    
    WWt = zeros(r, r)
    VWWt = zeros(m, r)
    
    VtX = zeros(r, n)
    
    VtV = zeros(r ,r)
    VtVW = zeros(r,n)
    #B = zeros(X)
    
    VW = zeros(X)
    X_VW = zeros(X)
    
    for iter = 1:maxiter

        V .= V .* A_mul_Bt!(XWt, X, W)./ A_mul_B!(VWWt, V, A_mul_Bt!(WWt, W, W))
        #to update matrix W
       
        W .= W .* At_mul_B!(VtX, V, X) ./ A_mul_B!(VtVW, At_mul_B!(VtV, V, V), W)
        # if reaches maxiter (and not satisfy the tolerance, stop with error message)
        
        X_VW .= X .- A_mul_B!(VW, V, W)
        
        f_normdiff_square = (vecnorm(X_VW))^2
        if f_normdiff_square < tol
           break
        end
    end
    # Output
    return V, W
end

nnmf3 (generic function with 1 method)

In [56]:
using BenchmarkTools
matrixt = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"));
@time nnmf3(matrixt, 50)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  4954k      0  0:00:01  0:00:01 --:--:-- 4952k


  5.850541 seconds (28 allocations: 16.612 MiB, 0.10% gc time)


([0.0363688 3.8291e-82 … 1.35504e-5 0.0384764; 0.0231779 6.6783e-68 … 0.00180108 0.0379306; … ; 0.0283529 0.00606037 … 0.000543965 6.85747e-24; 0.000937033 0.0080762 … 0.0411307 1.5269e-13], [2.02528e-83 1.46149e-63 … 3.57098e-6 2.10075e-56; 2.39701e-47 4.94378e-44 … 20.1848 18.9268; … ; 1.65677e-85 4.64916e-57 … 4.53552e-7 1.06863e-46; 1.25026e-58 4.19049e-68 … 3.79997e-12 1.64081e-64])

## Q2. Linear Mixed Models

Consider a linear mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```

**(1)** In matrix notation
$$
 \mathbf{y} = \mathbf{X} \beta + \mathbf{Z} \gamma + E,
$$
    Where $\varepsilon = (\epsilon_1, \ldots, \epsilon_n)^T, E \sim MVN(0,\sigma_0^2I_n)$. Since \gamma and E are independent normal-distributed random variables, and $\mathbf{X}$ and $\mathbf{Z}$ are known/fixed, $\mathbf{y}$ also follows normal distribution. And,
    
    $$ E(\mathbf{y}) = E(\mathbf{X} \beta) + E(\mathbf{Z} \gamma) + E(\varepsilon) = \mathbf{X} \beta + \mathbf{Z} E(\gamma) + E(\varepsilon) = \mathbf{X} \beta + \mathbf{0} + \mathbf{0} = \mathbf{X} \beta $$
    
$$
Var(\mathbf{y}) = Var(\mathbf{Z} \gamma) + Var(\varepsilon) = \mathbf{Z} Var(\gamma) \mathbf{Z}^T + Var(\varepsilon)=   \sigma_1^2 \mathbf{Z} \mathbf{Z}^T + \sigma_0^2 \mathbf{I}_n
$$

So $$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$

In [14]:
function logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number)
    n = length(y)
    Σ = σ0^2 + σ1^2 * Z * Z.'
    Σchol = cholfact(Symmetric(Σ))
    - (n/2) * log(2π) - (1/2) * logdet(Σchol) - (1/2) * dot(y, Σchol \ y)
end

logpdf_mvn (generic function with 1 method)

In [32]:
y = [1 ; 2; 3]
Z = [4 12 -16; 12 37 -43; -16 -43 98]
σ0 = 0.3
σ1 = 0.2
Σ = σ0^2 + σ1^2 * Z * Z.'
l = cholfact(Hermitian(Σ))

Symmetric(l[:L]);
l[:L]
#logpdf_mvn(y, Z, 0.3,0.2)
#mvn = MvNormal(Σ)



3×3 LowerTriangular{Float64,Array{Float64,2}}:
   4.09023   ⋅         ⋅     
  11.5617   0.947265   ⋅     
 -20.9841   2.97923   4.38636

In [24]:
@show x = rand(2,2)
x * x.'
x = [1.1 2.2 ; 2.2 1.1]
#t=cholfact(Symmetric(x))
A = [4 12 -16; 12 37 -43; -16 -43 98]
Achol = cholfact(A)
typeof(Achol)
@show Achol \ [2;3;4]
@show inv(A) * [2;3;4]

x = rand(2, 2) = [0.564948 0.637255; 0.307856 0.633021]
Achol \ [2; 3; 4] = [66.5, -18.0, 3.0]
inv(A) * [2; 3; 4] = [66.5, -18.0, 3.0]


3-element Array{Float64,1}:
  66.5
 -18.0
   3.0

In [1]:
using BenchmarkTools, Distributions

In [39]:
    srand(280)
    n, q = 1000, 50
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    #Σ = Symmetric(σ0^2 + σ1^2 * Z * Z.')
    #col = cholfact(Hermitian(Σ))
    #mvn = MvNormal(col) # MVN(0, Σ)
    # can be cholesky decomposed... why not P.D? 
    cholfact(Σ)
    #mvn = MvNormal(cholfact(Σ)[:L]+cholfact(Σ)[:L].')
    #y = rand(mvn) # generate one instance from MNV(0, Σ)
    #typeof(Symmetric(Σ))
#logpdf_mvn(y, Z, σ0, σ1)



Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[14.3609 1.82723 … 1.83222 1.2258; 0.0 12.2741 … -0.743993 -4.65323; … ; 0.0 0.0 … 0.513109 0.00599743; 0.0 0.0 … 0.0 0.516286]

In [2]:
srand(280) # seed

n = 1000
Σ = randn(n, n); Σ = Σ' * Σ + I # covariance matrix
y = rand(MvNormal(Σ)) # one randdom sample from N(0, Σ)


1000-element Array{Float64,1}:
 -24.0894   
 -57.7202   
 -29.7766   
 -26.5015   
 -27.2864   
   5.4884   
   0.300153 
 -54.2273   
  10.8998   
  21.636    
  17.3922   
 -15.7131   
  64.0949   
   ⋮        
 -30.0239   
 -38.9947   
   6.80544  
  43.8673   
 -18.4956   
  22.6042   
  -3.71631  
   0.356424 
  65.2194   
 -77.6038   
 -40.5898   
  -0.0629436